In [62]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
from  __builtin__ import any

In [213]:
def scrapeItems(urlList, itemName):
    if itemName=='desk':
        return
    itemDf = pd.DataFrame(columns=['Brand', 'Price', 'Image', 'Length', 'Width', 'Height', 'Seller', 'ProductLink'])

#     urlList = ["https://www.ikea.com/us/en/p/friheten-sleeper-sectional-3-seat-w-storage-skiftebo-blue-s69297560/", "https://www.ikea.com/us/en/p/karlstad-sofa-knisa-light-gray-s39132702/", "https://www.ikea.com/us/en/p/kivik-sofa-hillared-with-chaise-hillared-anthracite-s69193666/", "https://www.ikea.com/us/en/p/hemnes-daybed-with-3-drawers-2-mattresses-white-meistervik-firm-s59186119/", "https://www.ikea.com/us/en/p/knislinge-sofa-idhult-black-80278923/", "https://www.ikea.com/us/en/p/faerloev-sleeper-sofa-flodafors-white-s09252753/", "https://www.ikea.com/us/en/p/sandbacken-sleeper-sectional-3-seat-lofallet-beige-s89217906/"]
    for url in urlList:
        print(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        brand = soup.findAll("span", {"class":"product-pip__name"})[0].contents[0]
        price = soup.findAll("span", {"class":"product-pip__price__value"})[0].contents[0]
        #Contains all dimension-names from product description
        dimensions_desc = soup.findAll("dt", {"class":"product-pip__definition-list-item"})
        
        dimensions_desc_str=[str(dimensions_desc[x].contents[0]).lower().split(":")[0] for x in range(len(dimensions_desc))]
        
        imgLink = soup.findAll("img", {"class":""})[0]['src']
        #Contains all dimension-values from product description
        dimensions = soup.findAll("dd", {"class":"product-pip__definition-list-item"})
        print(dimensions_desc_str)
        
        lStr="length"
        wStr = "width"
        hStr = "height"
        
        #L,W,D
#         if "length" in dimensions_desc_str and "width" in dimensions_desc_str and "depth" in dimensions_desc_str:
        if any("length" in x for x in dimensions_desc_str) and any("width" in x for x in dimensions_desc_str) and any("depth" in x for x in dimensions_desc_str):
            hStr="depth"
            
        if any("thickness" in x for x in dimensions_desc_str) and any("width" in x for x in dimensions_desc_str) and any("depth" in x for x in dimensions_desc_str):
            hStr="thickness"
            
        if any("thickness" in x for x in dimensions_desc_str) and any("width" in x for x in dimensions_desc_str) and any("height" in x for x in dimensions_desc_str):
            lStr="thickness"
            
        if any("thickness" in x for x in dimensions_desc_str) and any("width" in x for x in dimensions_desc_str) and any("length" in x for x in dimensions_desc_str):
            hStr="thickness"
        #H,W,D
#         elif "height" in dimensions_desc_str and "width" in dimensions_desc_str and "depth" in dimensions_desc_str:
        elif any("height" in x for x in dimensions_desc_str) and any("width" in x for x in dimensions_desc_str) and any("depth" in x for x in dimensions_desc_str):
            lStr="depth"
        #L,D,H
#         elif "length" in dimensions_desc_str and "depth" in dimensions_desc_str and "height" in dimensions_desc_str:
        elif any("length" in x for x in dimensions_desc_str) and any("depth" in x for x in dimensions_desc_str) and any("height" in x for x in dimensions_desc_str):
            wStr="depth"
        
        print(lStr,wStr,hStr)
        #Finding All height-related dimensions
        try:
            length_index = []
            for dim in range(0, len(dimensions_desc)):
                if(lStr in dimensions_desc[dim].contents[0].lower()): #height
                    length_index.append(dim)

            #Finding max of given heights. Seemed in sync with most of the images 
            #with minute variations in the exceptions.
            lengths = []
            for index in length_index:
                lengths.append(dimensions[index].contents[0])

            length = max(lengths)
    #         print(length)

            width_index = []
            for dim in range(0, len(dimensions_desc)):
                if(wStr in dimensions_desc[dim].contents[0].lower()):
                    width_index.append(dim)
            widths = []
            for index in width_index:
                widths.append(dimensions[index].contents[0])

            width = max(widths)
    #         print(width)

            height_index = []
            for dim in range(0, len(dimensions_desc)):
                if(hStr in dimensions_desc[dim].contents[0].lower()):
                    height_index.append(dim)
            heights = []
            for index in height_index:
                heights.append(dimensions[index].contents[0])

            height = max(heights)
    #         print(height)
        except:
            continue

        productLink=url
        print(brand, price, imgLink, length, width, height, productLink)
        itemDf = itemDf.append({"Brand":brand, "Price":price, "Image":imgLink, "Length":length, "Width":width, "Height":height, "Seller":"IKEA", "ProductLink":productLink}, ignore_index=True)

#     itemDf
    
    filename = itemName+".csv"
    itemDf.to_csv(filename)

In [214]:
fourProducts = pd.read_csv("FourProductsURL.csv")
noOfItems = len(fourProducts)
items = list(fourProducts)
# print(items)
#for itemNumber in range(len(items)):
#     itemDf=fourProducts[items[itemNumber]].dropna()
scrapeItems(fourProducts['table'].dropna(),"table")

https://www.ikea.com/us/en/p/ingatorp-ingolf-table-and-4-chairs-black-nolhaga-gray-beige-s99254173/
['length', 'max. length', 'width', 'height']
length width height
INGATORP / INGOLF $547.00 https://www.ikea.com/PIAimages/0737940_PE741266_S5.JPG?f=s 61 " 34 1/4 " 29 1/8 " https://www.ikea.com/us/en/p/ingatorp-ingolf-table-and-4-chairs-black-nolhaga-gray-beige-s99254173/
https://www.ikea.com/us/en/p/ekedalen-extendable-table-brown-80340835/
['min. length', 'max. length', 'width', 'height']
length width height
EKEDALEN $119.00 https://www.ikea.com/PIAimages/0736972_PE740836_S5.JPG?f=s 47 1/4 " 27 1/2 " 29 1/2 " https://www.ikea.com/us/en/p/ekedalen-extendable-table-brown-80340835/
https://www.ikea.com/us/en/p/ekedalen-extendable-table-brown-30340809/
['min. length', 'max. length', 'width', 'height']
length width height
EKEDALEN $183.00 https://www.ikea.com/PIAimages/0736962_PE740826_S5.JPG?f=s 70 7/8 " 31 1/2 " 29 1/2 " https://www.ikea.com/us/en/p/ekedalen-extendable-table-brown-3034080

In [167]:
if "height" in "ax. height":
    print("Y")

Y
